# Enterprise RAG System - Evaluation

This notebook covers evaluation metrics:
1. **Faithfulness**: Every claim traceable to context
2. **Recall**: Relevant information retrieved
3. **Precision**: Minimal irrelevant content
4. **Citation Accuracy**: Correct section/page references
5. **Failure Handling**: Proper "not found" responses

In [ ]:
import sys
sys.path.insert(0, '..')

from config.settings import settings
settings.initialize()

## Evaluation Framework

### Metrics Definition

| Metric | Description | Target |
|--------|-------------|--------|
| Faithfulness | % of claims grounded in context | >95% |
| Retrieval Recall | % of relevant chunks retrieved | >80% |
| Citation Accuracy | % of citations with correct page | >90% |
| "Not Found" Precision | Correct identification of unanswerable | >95% |

In [ ]:
from dataclasses import dataclass
from typing import List, Dict, Any

@dataclass
class EvaluationResult:
    """Result of evaluating a single query."""
    query: str
    expected_answer: str
    actual_answer: str
    
    # Metrics
    faithfulness_score: float  # 0-1
    retrieval_recall: float    # 0-1
    citation_accuracy: float   # 0-1
    
    # Flags
    is_grounded: bool
    has_hallucination: bool
    correct_not_found: bool

print("Evaluation framework defined")

## Test Cases

### Answerable Questions (should find answer)

In [ ]:
answerable_queries = [
    {
        "query": "What are the main risk factors mentioned in the filing?",
        "expected_section": "Item 1A",
        "expected_keywords": ["risk", "uncertainty", "factors"]
    },
    {
        "query": "What was the total revenue for the fiscal year?",
        "expected_section": "Item 7",
        "expected_keywords": ["revenue", "sales", "$"]
    },
    {
        "query": "Describe the company's business model.",
        "expected_section": "Item 1",
        "expected_keywords": ["business", "products", "services"]
    }
]

print(f"Defined {len(answerable_queries)} answerable test cases")

### Unanswerable Questions (should return "not found")

In [ ]:
unanswerable_queries = [
    "What is the weather forecast for tomorrow?",
    "Who won the Super Bowl last year?",
    "What is the capital of France?",
    "What did the CEO have for breakfast?"
]

print(f"Defined {len(unanswerable_queries)} unanswerable test cases")

## Evaluation Functions

In [ ]:
import re

def check_faithfulness(answer: str, context_chunks: list) -> float:
    """
    Check if answer claims are grounded in context.
    
    Simple approach: Check if key phrases in answer
    appear in source chunks.
    """
    if not context_chunks:
        return 0.0
    
    # Combine context
    context_text = " ".join(c.text for c in context_chunks).lower()
    
    # Extract key phrases from answer (simplified)
    # In production, use NLP to extract claims
    answer_sentences = answer.split('.')
    
    grounded = 0
    total = 0
    
    for sentence in answer_sentences:
        sentence = sentence.strip().lower()
        if len(sentence) < 10:
            continue
        
        total += 1
        # Check if key words appear in context
        words = [w for w in sentence.split() if len(w) > 4]
        matches = sum(1 for w in words if w in context_text)
        
        if matches / max(len(words), 1) > 0.3:
            grounded += 1
    
    return grounded / max(total, 1)


def check_is_not_found(answer: str) -> bool:
    """Check if answer indicates 'information not found'."""
    patterns = [
        r"not present in the provided documents",
        r"information is not available",
        r"cannot find",
        r"no information about"
    ]
    
    answer_lower = answer.lower()
    return any(re.search(p, answer_lower) for p in patterns)


def check_citation_accuracy(citations: list, expected_section: str) -> float:
    """Check if citations reference expected section."""
    if not citations:
        return 0.0
    
    matches = sum(
        1 for c in citations 
        if expected_section.lower() in c.get('section', '').lower()
    )
    
    return matches / len(citations)

print("Evaluation functions defined")

## Run Evaluation

In [ ]:
from src.pipeline.query import QueryPipeline

# Initialize pipeline
# pipeline = QueryPipeline()

def evaluate_query(query: str, expected_section: str = None):
    """
    Evaluate a single query.
    
    Returns evaluation metrics.
    """
    # Run query
    # response = pipeline.query(query, verbose=False)
    
    # Evaluate
    # faithfulness = check_faithfulness(response.answer, response.source_chunks)
    # is_not_found = check_is_not_found(response.answer)
    # citation_acc = check_citation_accuracy(response.citations, expected_section or '')
    
    # return {
    #     'query': query,
    #     'answer': response.answer,
    #     'faithfulness': faithfulness,
    #     'confidence': response.confidence,
    #     'is_not_found': is_not_found,
    #     'citation_accuracy': citation_acc,
    #     'num_sources': len(response.source_chunks)
    # }
    
    print(f"Evaluation function ready (uncomment when documents ingested)")

evaluate_query("test")  # Demo call

## Known Failure Cases

### 1. Complex Numerical Reasoning
The system may struggle with multi-step calculations that require combining data from multiple tables.

### 2. Temporal Comparisons
Comparisons across multiple years require understanding time context.

### 3. Implicit Information
Information that must be inferred (not explicitly stated) may not be retrieved.

### 4. Very Long Documents
Section detection may fail on non-standard PDF layouts.

In [ ]:
failure_cases = [
    {
        "type": "Complex Reasoning",
        "example": "Calculate the year-over-year growth rate for each segment",
        "reason": "Requires math on retrieved table data"
    },
    {
        "type": "Cross-Document",
        "example": "Compare Apple and Microsoft's revenue",
        "reason": "Requires querying multiple documents"
    },
    {
        "type": "Implicit",
        "example": "Is the company in financial trouble?",
        "reason": "Requires inference and judgment"
    }
]

print("Known failure cases:")
for fc in failure_cases:
    print(f"\n{fc['type']}:")
    print(f"  Example: {fc['example']}")
    print(f"  Reason: {fc['reason']}")

## Recommendations for Production

1. **Use RAGAS or similar framework** for automated evaluation
2. **Create ground truth dataset** for your specific documents
3. **Monitor confidence scores** in production
4. **Log "not found" responses** for analysis
5. **A/B test** different reranker thresholds